The following code queries all bakers in the 6 previous cycles and their versions.

To run, press Crtl + Enter twice.

In [64]:
import requests
import json
import datetime
# get current cycle
current_cycle = int(requests.get("https://api.tzstats.com/explorer/cycle/head").json()["cycle"])

print("Starting queries... it will take a while")
# make list of active bakers
bakers_dict = {}
for cycle in range(current_cycle - 5, current_cycle+1):
    bakers_json = requests.get(f"https://api.tzstats.com/tables/income?&columns=address,rolls&cycle={cycle}")
    bakers = bakers_json.json()
    for baker in bakers:
        if baker[0] not in bakers_dict:
            bakers_dict[baker[0]] = { "rolls": baker[1] }

# uncomment line below to make the list smaller while developing
bakers_dict = { k:v for k,v in bakers_dict.items() if k in list(bakers_dict)[:100]}

# query baker alias from tzkt (tzstats does not have it)

tzkt_account_json = requests.get("https://api.tzkt.io/v1/delegates/?select=address,alias,balance&limit=10000")
bakers_alias={account['address']:account['alias'] for account in tzkt_account_json.json()}

# hardcoded version numbers ... not sure where to look
version_nums = { "d74e7b6d":"v8.1",
               "8e3af918": "v8.2",
               "ce365456": "v7.5",
               "e69b63f1": "v7.4",
               "567be003": "v8.0",
               "c90787b9": "v8.0-rc1",
               "13067b95": "mainnet-2020-june"}

for baker_idx, baker in enumerate(list(bakers_dict)):
    baker_version = requests.get(f"https://api.tzstats.com/explorer/account/{baker}").json()["baker_version"]
    # query baker alias from tzkt (tzstats does not have it)

    if baker_idx % 10 == 0:
        print(f"{len(bakers_dict) - baker_idx} bakers left to query...")
    bakers_dict[baker]["version"] = version_nums.get(baker_version, baker_version)
    bakers_dict[baker]["alias"] = bakers_alias.get(baker)
    
#Organize upgrade results in a table compatible with Pandas DataFrame
results_pn = {'address':[],
              'alias': [],
              'rolls':[],
              'version':[]
                       }


# Populate upgrade reliability results per baker in a separate dictionary
for k,v in bakers_dict.items():
    results_pn['address'].append(k)
    results_pn['alias'].append(v['alias'])
    results_pn['rolls'].append(v['rolls'])
    results_pn['version'].append(v['version'])
    
print("all done!")

Starting queries... it will take a while
100 bakers left to query...
90 bakers left to query...
80 bakers left to query...
70 bakers left to query...
60 bakers left to query...
50 bakers left to query...
40 bakers left to query...
30 bakers left to query...
20 bakers left to query...
10 bakers left to query...
all done!


In [69]:
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.DataFrame(results_pn)
df = df.sort_values(by=['rolls'], ascending=False)
from IPython.display import display, HTML

display(df)

,address,alias,rolls,version
0,tz3RDC3Jdn4j15J7bBHZd29EUee9gVB1CxD9,Foundation baker 6,2864,v8.1
3,tz3bTdwZinP8U1JmSweNzVKhmwafqWmFWRfk,Foundation baker 2,2862,v8.1
4,tz3NExpXn9aPNZPorRE4SdjJ2RGrfbJgMAaV,Foundation baker 8,2862,v8.1
2,tz3RB4aoyjov4KEVRbuhvQ1CKJgBJMWhaeB8,Foundation baker 7,2861,v8.1
7,tz3VEZ4k6a4Wx42iyev6i2aVAptTRLEAivNN,Foundation baker 4,2860,v8.1
6,tz3WMqdzXqRWXwyvj5Hp2H7QEepaUuS7vd9K,Foundation baker 3,2859,v8.1
1,tz3bvNMQ95vfAYtG8193ymshqjSvmxiCUuR5,Foundation baker 1,2854,v8.1
5,tz3UoffC7FG7zfpmvmjUmUeAaHvzdcUvAj6r,Foundation baker 5,2675,v8.1
77,tz1WCd2jm4uSt4vntk4vSuUWoZQGhLcDuR9q,Happy Tezos,921,v7.4
94,tz1KfEsrtDaA1sX7vdM4qmEPWuSytuqCDp5j,XTZMaster,894,v8.1
